In [2]:
from op_analytics.datasources.github.dataaccess import Github
from op_analytics.coreutils.duckdb_inmem.client import init_client

prs_table = Github.PRS.read()
ctx = init_client()

# Get a count by date.
print(ctx.client.sql(f"SELECT dt, count(*) FROM {prs_table} GROUP BY 1 ORDER BY 1"))

# Get the raw PR data.
prs_df = ctx.client.sql(f"SELECT dt, repo, number FROM {prs_table}").pl()

2025-01-28 14:14:45 [info     ] Reading data from 'github/github_prs_v1' with filters min_date=None, max_date=None, date_range_spec=None filename=dailydata.py lineno=187 process=58371
2025-01-28 14:14:47 [info     ] registered view: 'github_github_prs_v1' using uri wildcard 'gs://oplabs-tools-data-sink/github/github_prs_v1/dt=*/out.parquet' filename=client.py lineno=291 process=58371
┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ github_github_prs_v1 │
└──────────────────────┘

┌────────────┬──────────────┐
│     dt     │ count_star() │
│    date    │    int64     │
├────────────┼──────────────┤
│ 2020-12-31 │          260 │
│ 2021-12-31 │         1316 │
│ 2022-12-31 │         2352 │
│ 2023-12-31 │         3961 │
│ 2025-01-13 │         6411 │
│ 2025-01-15 │          420 │
│ 2025-01-16 │          431 │
│ 2025-01-17 │          443 │
│ 2025-01-18 │          427 │
│ 2025-01-19 │          438 │
│ 2025-01-20 │          436 │
│ 2025-01-21

In [4]:
# Helper functions.

import os
from unittest.mock import patch

import polars as pl

from op_analytics.coreutils.partitioned import dailydata
from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.time import date_fromstr
from op_analytics.datasources.github.activity.githubapi import OptimismRepo, bulk_fetch_reviews
from op_analytics.datasources.github.dataaccess import Github


def fetch_backfill_reviews(target_date: str, pull_requests_df: pl.DataFrame):
    """Fetch the backfill reviews dataframe."""
    
    target_dt = date_fromstr(target_date)
    dfs = []
    for (dt, repo), group_df in pull_requests_df.sort("dt", "repo").group_by("dt", "repo"):
        if dt != target_dt:
            continue

        print(dt, repo, len(group_df))
        repo_obj = OptimismRepo(repo)
        pr_numbers = group_df["number"].to_list()
        pr_reviews = bulk_fetch_reviews(repo_obj, pr_numbers)

        extracols = dict(repo=pl.lit(repo), dt=pl.lit(target_date))
        dfs.append(pr_reviews.with_columns(**extracols))

    return pl.concat(dfs)


def overwrite_reviews(new_reviews_df):
    """Overwrite GCS with the backfill reviews df."""
    
    # Required to allow writing data to GCS from local.
    os.environ["ALLOW_WRITE"] = "true"

    # Override location for local. Othersise It will default to DataLocation.LOCAL when running from laptop.
    def mock_location():
        return DataLocation.GCS

    with patch.object(dailydata, "determine_location", mock_location):
        Github.PR_REVIEWS.write(
            dataframe=new_reviews_df,
            sort_by=["repo", "pr_number", "submitted_at"],
        )

In [6]:
# The various original backfill "dt"s
# The first few dates have a lot of historical data, so I ran them by hand one by one.

# target_date = "2020-12-31"
# target_date = "2021-12-31"
# target_date = "2022-12-31"
# target_date = "2023-12-31"
# target_date = "2025-01-13"
# target_date = "2025-01-15"
reviews_df = fetch_backfill_reviews(target_date="2025-01-16", pull_requests_df=prs_df)


2025-01-16 design-docs 27
2025-01-28 14:16:04 [info     ] fetching reviews for 27 prs    filename=githubapi.py lineno=99 process=58371
2025-01-28 14:16:06 [info     ] fetching data for prs. completed 1 of 27 filename=githubapi.py lineno=122 process=58371
2025-01-28 14:16:15 [info     ] fetching data for prs. completed 6 of 27 filename=githubapi.py lineno=122 process=58371
2025-01-28 14:16:25 [info     ] fetching data for prs. completed 11 of 27 filename=githubapi.py lineno=122 process=58371
2025-01-28 14:16:35 [info     ] fetching data for prs. completed 16 of 27 filename=githubapi.py lineno=122 process=58371
2025-01-28 14:16:45 [info     ] fetching data for prs. completed 21 of 27 filename=githubapi.py lineno=122 process=58371
2025-01-28 14:16:54 [info     ] fetching data for prs. completed 26 of 27 filename=githubapi.py lineno=122 process=58371
2025-01-28 14:16:56 [info     ] fetching data for prs. completed 27 of 27 filename=githubapi.py lineno=122 process=58371
2025-01-16 specs 40


In [7]:
len(reviews_df)

1731

In [8]:
overwrite_reviews(new_reviews_df=reviews_df)

2025-01-28 14:30:42 [debug    ] Found vault variable GOOGLE_SERVICE_ACCOUNT (has JSON key) filename=gcpauth.py lineno=18 process=58371 root=github/github_pr_reviews_v2
2025-01-28 14:30:42 [info     ] Initialized gcsfs client for bucket=gs://oplabs-tools-data-sink filename=gcs_parquet.py lineno=32 process=58371 root=github/github_pr_reviews_v2
2025-01-28 14:30:42 [info     ] done writing 1.7Krows 88.2KB   filename=gcs_parquet.py lineno=57 maxrss=301973504 path=oplabs-tools-data-sink/github/github_pr_reviews_v2/dt=2025-01-16/out.parquet process=58371 root=github/github_pr_reviews_v2 rows=1731 size=88172
2025-01-28 14:30:42 [info     ] memory usage                   filename=gcs_parquet.py lineno=65 max_rss=301.97 process=58371 root=github/github_pr_reviews_v2
2025-01-28 14:30:42 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=36 process=58371 root=github/github_pr_reviews_v2
2025-01-28 14:30:43 [debug    ] initialized OPLABS Clickhouse client. filename=cli

In [ ]:
# The last dates I ran in a loop since each one of them goes much faster.

for val in [
    "2025-01-17",
    "2025-01-18",
    "2025-01-19",
    "2025-01-20",
    "2025-01-21",
    "2025-01-22",
    "2025-01-23",
    "2025-01-24",
    "2025-01-25",
    "2025-01-26",
    "2025-01-27",
    "2025-01-28",
]:
    reviews_df = fetch_backfill_reviews(target_date=val, pull_requests_df=prs_df)
    print("\n---")
    print(val, len(reviews_df))
    overwrite_reviews(new_reviews_df=reviews_df)